In [1]:
import os
import cv2 as cv
import numpy as np
from numpy.polynomial import polynomial as P
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from matplotlib import pyplot as plt
from tqdm.auto import tqdm
%matplotlib inline
%load_ext cython

In [2]:
%%cython -a
import cv2 as cv
import numpy as np
cimport numpy as np
from numpy.polynomial import polynomial as P
import cython
cimport cython

cdef int[:] hat_weight = np.concatenate((np.arange(128, dtype=np.intc), np.arange(128, dtype=np.intc)[::-1]))

@cython.nonecheck(False)
@cython.boundscheck(False)
@cython.wraparound(False)  # turn off negative index wrapping for entire function
cpdef float [:,:,:,:] GetLocalShadowParameter(shadow, 
                                              shadow_free, 
                                              int ksize=5, int deg=1):
    """
    Calculate shadow parameters based on neighboring region.
    """
    
    cdef int border = (ksize-1)//2  # ksize must be an odd number
    bordered_shadow = \
        cv.copyMakeBorder(shadow, border, border, border, border, cv.BORDER_REPLICATE)
    bordered_shadow_free = \
        cv.copyMakeBorder(shadow_free, border, border, border, border, cv.BORDER_REPLICATE)

    cdef float[:,:,:,:] sp = np.empty((shadow.shape[0], shadow.shape[1], 3, (deg+1)), dtype=np.float32)
    cdef float[:, :] weight = np.ones((ksize, ksize), dtype=np.float32)
    cdef int i, j, k
    for i in range(1, border+1):
        for j in range(i, ksize-i):
            for k in range(i, ksize-i):
                weight[j, k] += 2

    cdef int r, c, channel
    cdef unsigned char[:] x, y
    cdef float[:] w = np.empty(ksize*ksize, dtype=np.float32)
    cdef float[:] coef
    for r in range(shadow.shape[0]):
        for c in range(shadow.shape[1]):
            for channel in range(3):
                x = bordered_shadow[r:r+ksize, c:c+ksize, channel].ravel()
                y = bordered_shadow_free[r:r+ksize, c:c+ksize, channel].ravel()
                for i in range(ksize):
                    for j in range(ksize):
                        w[i*ksize+j] = weight[i, j] * hat_weight[y[i*ksize+j]] 
                coef = (P.polyfit(x, y, deg, full=False, w=w)).astype(np.float32)
                sp[r, c, channel] = coef

    # if show_plot:
    #     plt.scatter(x, y, marker='.', label='data')
    #     plt.plot(x, y_pred, 'b-', label='linear fit')
    #     plt.legend(loc='lower right')
    #     plt.title('Linear regression')
    #     if savepath is not None:
    #         plt.savefig(savepath)
    #     plt.show()

    return sp

In [3]:
def test_local(dataset, file, save_sp=True, save_img=True):
    img = cv.imread(os.path.join(".", dataset, dataset+"_A", file), cv.IMREAD_COLOR)
    img_gt = cv.imread(os.path.join(".", dataset, dataset+"_C_fixed_official", file), cv.IMREAD_COLOR)
    sp = GetLocalShadowParameter(img, img_gt)
    if save_sp:
        sp_dir = os.path.join(".", dataset, "sp")
        if not (os.path.exists(sp_dir) and os.path.isdir(sp_dir)):
            os.makedirs(sp_dir, exist_ok=True)
        np.save(os.path.join(sp_dir, file.split('.')[0]), sp)
    result = np.clip((sp[..., 0] + sp[..., 1] * img), 0, 255).astype(np.uint8)
    # plt.figure()
    # plt.imshow(cv.cvtColor(result, cv.COLOR_BGR2RGB))
    # plt.close()
    if save_img:
        img_dir = os.path.join(".", dataset, "result_C_fixed_official")
        if not (os.path.exists(img_dir) and os.path.isdir(img_dir)):
            os.makedirs(img_dir, exist_ok=True)
        cv.imwrite(os.path.join(img_dir, file), result)
    return result

In [ ]:
for current_set in ["test", "train"]:
    directory = os.path.join(current_set, current_set+"_A")
    filenames = os.listdir(directory)
    filenames.sort()
    for f in tqdm(filenames):
        test_local(current_set, f)
        
# test_local("1-1.png")

/home/nathan/.pyenv/versions/shadow-removal/lib/python3.8/site-packages/numpy/polynomial/polynomial.py:1336: RankWarning: The fit may be poorly conditioned
  return pu._fit(polyvander, x, y, deg, rcond, full, w)


In [5]:
%%cython -a
import cv2 as cv
import numpy as np
from numpy.polynomial import polynomial as P
hat_weight = np.concatenate((np.arange(128), np.arange(128)[::-1]))
def GetLocalShadowParameterPy(shadow, shadow_free, ksize=5, deg=1):
    """ 
    Calculate shadow parameters based on neighboring region.
    """

    border = (ksize-1)//2  # ksize must be an odd number
    bordered_shadow = cv.copyMakeBorder(shadow, border, border, border, border, cv.BORDER_REPLICATE)
    bordered_shadow_free = cv.copyMakeBorder(shadow_free, border, border, border, border, cv.BORDER_REPLICATE)

    sp = np.empty((shadow.shape[0], shadow.shape[1], 3, (deg+1)), dtype=np.float32)
    weight = np.ones((ksize, ksize))
    for i in range(1, border+1):
        weight[i:ksize-i, i:ksize-i] += 2
    weight = weight.ravel()

    for r in (range(shadow.shape[0])):
        for c in range(shadow.shape[1]):
            for channel in range(3):
                x = bordered_shadow[r:r+ksize, c:c+ksize, channel].ravel()
                y = bordered_shadow_free[r:r+ksize, c:c+ksize, channel].ravel()
                w = weight * hat_weight[y]
                coef, stats = P.polyfit(x, y, deg, full=True, w=w)
                sp[r, c, channel] = coef

    # if show_plot:
    #     plt.scatter(x, y, marker='.', label='data')
    #     plt.plot(x, y_pred, 'b-', label='linear fit')
    #     plt.legend(loc='lower right')
    #     plt.title('Linear regression')
    #     if savepath is not None:
    #         plt.savefig(savepath)
    #     plt.show()

    return sp